# Sestavimo Agenta
V tem zvezku bomo skupaj sesatvili spraševanje po strukturiranih in nestrukturiranih podatkih in to zapakirali v agenta.
Agent je v LangChain-u opisan kot:\
\
"The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order." 

Preko primera bomo predstavili nekatere ključne koncepte:
* Agent - Veriga odgovorna za izbiro naslednjega koraka.
* Tools - Fukcije, katere lahko agent uporablja.
* Toolkits - Zbirka relevantih orodij (veliko pred definiranih).
* AgentExecutor - runtime agenta (reševanje napak, logging, ...).


In [ ]:
%%capture
!pip install langchain langchain_community langchain_openai python-dotenv openai faiss-cpu

In [ ]:
import os
import requests
from typing import Optional

import dotenv

from langchain.agents import AgentType, initialize_agent, create_react_agent
from langchain_openai import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.utilities import SQLDatabase
from langchain.memory import ConversationBufferMemory
from langchain.tools.base import BaseTool
from langchain.tools.sql_database.tool import BaseSQLDatabaseTool
from langchain.callbacks.manager import CallbackManagerForToolRun
from langchain.vectorstores import FAISS
from langchain.schema.vectorstore import VectorStoreRetriever
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
if not dotenv.load_dotenv():
    os.environ["DB_CONNECT"] = "postgres-uri"
    os.environ["OPENAI_API_KEY"] = "openai-key"

#### Inicializacija komponent

In [ ]:
llm = ChatOpenAI(temperature=0, model="gpt-4-turbo")

db = SQLDatabase.from_uri(os.getenv("DB_CONNECT"))
sql_toolkit = SQLDatabaseToolkit(db=db, llm=llm)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

vectorstore = FAISS.load_local("../Data/nba_rules_faiss", embeddings=OpenAIEmbeddings(), allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

#### Definicija orodij

In [ ]:
class QuerySQLDataBaseTool(BaseSQLDatabaseTool, BaseTool):
    """Tool for querying a SQL database."""

    name: str = "sql_db_query"
    description: str = """
    Input to this tool is a detailed and correct PostgreSQL query, output is a result from the database.
    If the query is not correct, an error message will be returned.
    If an error is returned, rewrite the query, check the query, and try again.
    """

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Execute the query, return the results or an error message."""
        return self.db.run_no_throw(query)

In [ ]:
class CheckSchedule(BaseTool):
    """Tool for querying a SQL database."""

    name: str = "check_schedule"
    description: str = """
    Input to this tool is date in the format YYYY-MM-DD, output is list of games with the corresponding ID and teams.
    If the date is not correct, an error message will be returned. If there are no games on the date, an empty list will be returned.
    """

    def _run(
        self,
        date: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Execute the query, return the results or an error message."""
        response = requests.get(f"http://0.0.0.0:8000/schedule/{date}").json()
        return f"The available games are {response}. Respond to the user with the appropriate matchups and the corresponding IDs."

In [ ]:
class ReserveSeat(BaseTool):
    """Tool for querying a SQL database."""

    name: str = "reserve_seat"
    description: str = """
    Input to this tool is a valid game ID obtained from 'check_schedule' tool, output is a string.
    If the ID is not correct, an error message will be returned. Make sure to use the ID from the list of games on the date you want to attend.
    """

    def _run(
        self,
        ID: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Execute the query, return the results or an error message."""
        response = requests.post(f"http://0.0.0.0:8000/reserve-seat/{ID}").json()

        if "detail" in response:
            return response["detail"]
        elif "seat_number" in response:
            return f"Your seat has been reserved. Your seat number is {response['seat_number']}. Please arrive 30 minutes before the game starts. Your reservation ID is {response['reservation_id']}"
        else:
            return "Something went wrong. Please try again."

In [ ]:
class WikiAndRulebook(BaseTool):
    """Tool for querying a SQL database."""

    name: str = "rulebook_and_wiki_info"
    description: str = """
    Input to this tool is a string with a detailed description of the topic of interest, output is a string providing potentially relevant results.
    Use this tool for info about the NBA rules and general info not provided in the database.
    """
    retriever: VectorStoreRetriever = None

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Execute the query, return the results or an error message."""
        relevant_documents = retriever.get_relevant_documents(query)
        return f"""The retrieved documents are encased in triple ticks:
    ```
    {relevant_documents}
    ```
    Use the information provided in the documents if its relevant."""

In [ ]:
tools = [
    QuerySQLDataBaseTool(db=db),
    CheckSchedule(),
    ReserveSeat(),
    WikiAndRulebook(retriever=retriever),
]

#### Sistemsko sporočilo za Gospoda NBA

In [ ]:
system_message = """You are an NBA journalist called Mr. NBA. Current date is 2023-11-21.
You are designed to assist to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics.
You are friendly and helpful and will try to answer any questions you are asked that are related to the NBA.
Given an input question, you have multiple tools to help you answer the question. When interacting with the PostgreSQL database create a syntactically correct PostgreSQL query to run.
Wrap each column name in double quotes (") to denote them as delimited identifiers. You MUST double check your query before executing it.
If you get an error while executing a query, rewrite the query and try again. DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
Do not return ID's of teams and players, instead return their names. When talking about a game always mention between which teams the game was played.

If the question does not seem related to the NBA and the information provided by the tools can't help you reply with 'Sorry mister I just don't know.'.
If you are not confident in your answer or if the data is not available, reply with 'Sorry mister I am not sure.'.
Only use the information returned by the below tools to construct your final answer.

The follwoing tables are available in the datatabase:
TABLE general_game_details (\n\tgame_date_est TEXT, \n\tgame_id BIGINT, \n\thome_team_id BIGINT, \n\tvisitor_team_id BIGINT, \n\tseason BIGINT, \n\tpoints_scored_home_team DOUBLE PRECISION, \n\tfield_goal_percentage_home_team DOUBLE PRECISION, \n\tfree_throw_percentage_home_team DOUBLE PRECISION, \n\tthree_point_percentage_home_team DOUBLE PRECISION, \n\tassists_home_team DOUBLE PRECISION, \n\trebounds_home_team DOUBLE PRECISION, \n\tpoints_scored_away_team DOUBLE PRECISION, \n\tfield_goal_percentage_away_team DOUBLE PRECISION, \n\tfree_throw_percentage_away_team DOUBLE PRECISION, \n\tthree_point_percentage_away_team DOUBLE PRECISION, \n\tassists_away_team DOUBLE PRECISION, \n\trebounds_away_team DOUBLE PRECISION, \n\thome_team_wins BIGINT\n)

TABLE player_game_details (\n\tgame_id BIGINT, \n\tteam_id BIGINT, \n\tteam_abbreviation TEXT, \n\tteam_city TEXT, \n\tplayer_id BIGINT, \n\tplayer_name TEXT, \n\tnickname TEXT, \n\tminutes_played TEXT, \n\tfield_goals_made DOUBLE PRECISION, \n\tfield_goals_attempted DOUBLE PRECISION, \n\tfield_goal_percentage DOUBLE PRECISION, \n\tthree_pointers_made DOUBLE PRECISION, \n\tthree_pointers_attempted DOUBLE PRECISION, \n\tthree_pointers_percentage DOUBLE PRECISION, \n\tfree_throws_made DOUBLE PRECISION, \n\tfree_throws_attempted DOUBLE PRECISION, \n\tfree_throws_percentage DOUBLE PRECISION, \n\toffensive_rebounds DOUBLE PRECISION, \n\tdefensive_rebounds DOUBLE PRECISION, \n\trebounds DOUBLE PRECISION, \n\tassists DOUBLE PRECISION, \n\tstalls DOUBLE PRECISION, \n\tblocked_shots DOUBLE PRECISION, \n\tturnovers DOUBLE PRECISION, \n\tpersonal_fouls DOUBLE PRECISION, \n\tpoints_scored DOUBLE PRECISION\n)

TABLE player_general_details (\n\tplayer_name TEXT, \n\tteam_id BIGINT, \n\tplayer_id BIGINT, \n\tseason BIGINT\n)

TABLE team_general_details (\n\tteam_id BIGINT, \n\tabbreviation TEXT, \n\tnickname TEXT, \n\tcity TEXT, \n\tarena TEXT, \n\tarena_capacity DOUBLE PRECISION, \n\towner TEXT, \n\tgeneral_manager TEXT, \n\theadcoach TEXT)

TABLE team_rankings (\n\tteam_id BIGINT, \n\tleague_id BIGINT, \n\tseason_id BIGINT, \n\tstandingsdate TEXT, \n\tconference TEXT, \n\tteam TEXT, \n\tgames_played BIGINT, \n\tgames_won BIGINT, \n\tgames_lost BIGINT, \n\twin_percentage DOUBLE PRECISION, \n\thome_record TEXT, \n\troad_record TEXT\n)
"""

#### Sestavljanje agenta

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
    agent_kwargs={"system_message": system_message},
    handle_parsing_errors=True,
)

### Primeri
Sesatvili smo NBA agenta, ki je sposoben odgovrajati na naša vprašanja. 

Čas za testiranje:

In [ ]:
agent_chain.invoke(input="Hey, what is your name?")["output"]

In [ ]:
agent_chain.invoke(
    input="What game did Luka Doncic have the highest three pointers percentage in?"
)["output"]

In [ ]:
agent_chain.invoke(input="What was his three point percentage?")["output"]

In [ ]:
agent_chain.invoke(input="Who did they play against?")["output"]

In [ ]:
agent_chain.invoke(input="Are there any Dalas games on 2023-12-12?")["output"]

In [ ]:
agent_chain.invoke(input="Make me a reservation for that game?")["output"]

In [ ]:
agent_chain.invoke(input="When did Nikola Jokic start playing in the NBA?")["output"]

In [ ]:
agent_chain.invoke(input="What was his best game in terms of three pointers percentage?")["output"]

In [ ]:
agent_chain.invoke(input="Which teams played in that game?")["output"]